# Pancreatic cancer

In [1]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt

# import booleanNetwork module from ./src
from BNMPy import booleanNetwork as bn
from BNMPy import BMatrix  

In [23]:
file = 'input_files/pancreatic_vundavilli_2020_fig3.txt'

equations = BMatrix.get_equations(file)
gene_dict = BMatrix.get_gene_dict(equations)
upstream_genes = BMatrix.get_upstream_genes(equations)

In [24]:
connectivity_matrix = BMatrix.get_connectivity_matrix(equations, upstream_genes, gene_dict)

In [25]:
truth_table = BMatrix.get_truth_table(equations, upstream_genes)

In [27]:
ngenes = len(equations)

In [28]:
# initial state
x0 = np.random.randint(2, size=ngenes) #random inital state 
x0 = np.array(x0)

In [29]:
%%time

network = bn.BooleanNetwork( ngenes , connectivity_matrix, truth_table, x0  ) # create a Boolean network object
noise_level = 0.05 # noise
y = network.update_noise ( noise_level  , 200000) # simulation with noise

CPU times: user 6.98 s, sys: 0 ns, total: 6.98 s
Wall time: 6.98 s


In [31]:
y.shape

(200001, 38)